https://www.climatologylab.org/terraclimate-variables.html

netCDF version 4 has many features not found in earlier versions of the library, such as hierarchical groups, zlib compression, multiple unlimited dimensions, and new data types.

In [1]:
#!pip install netCDF4
import netCDF4
import xarray as xr
import rasterio
from rasterio.transform import from_origin

In [2]:
# Open the downloaded NetCDF file
f = netCDF4.Dataset(r'C:\Users\taohuang\Downloads\TerraClimate_def_2024.nc', 'r')

print( f)

<class 'netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    method: These layers from TerraClimate were derived from the essential climate variables of TerraClimate. Water balance variables, actual evapotranspiration, climatic water deficit, runoff, soil moisture, and snow water equivalent were calculated using a water balance model and plant extractable soil water capacity derived from Wang-Erlandsson et al (2016).
    title: TerraClimate: monthly climate and climatic water balance for global land surfaces
    summary: This archive contains a dataset of high-spatial resolution (1/24°, ~4-km) monthly climate and climatic water balance for global terrestrial surfaces from 1958-2015. These data were created by using climatically aided interpolation, combining high-spatial resolution climatological normals from the WorldClim version 1.4 and version 2 datasets, with coarser resolution time varying (i.e. monthly) data from CRU Ts4.0 and JRA-55 to produce a month

In [3]:
print(f.variables.keys()) # get all variable names

dict_keys(['lat', 'lon', 'time', 'crs', 'def'])


In [4]:
cdef = f.variables['def']
print(cdef)

<class 'netCDF4.Variable'>
int16 def(time, lat, lon)
    _FillValue: -32768
    units: mm
    description: Climatic Water Deficit
    long_name: water_potential_evaporation_amount_minus_water_evaporation_amount
    standard_name: water_potential_evaporation_amount_minus_water_evaporation_amount
    missing_value: -32768
    dimensions: lon lat time
    grid_mapping: crs
    coordinate_system: WGS84,EPSG:4326
    scale_factor: 0.1
    add_offset: 0.0
    _Unsigned: false
unlimited dimensions: 
current shape = (12, 4320, 8640)
filling on


In [5]:
cdef.shape

(12, 4320, 8640)

In [6]:
time = f.variables['time']
print(time)

<class 'netCDF4.Variable'>
float64 time(time)
    description: days since 1900-01-01
    units: days since 1900-01-01 00:00:00
    long_name: time
    standard_name: time
    calendar: gregorian
unlimited dimensions: 
current shape = (12,)
filling on, default _FillValue of 9.969209968386869e+36 used


Masked arrays work with any type of data, not just with floating point.

In [7]:
def_last_layer = f.variables['def'][-1, :, :]    
print(def_last_layer)
print( type(def_last_layer) )
print( def_last_layer.dtype)               
print( def_last_layer.data.dtype)          # same as above
print( def_last_layer.mask.dtype)          # bool

[[-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 [-- -- -- ... -- -- --]
 ...
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]]
<class 'numpy.ma.MaskedArray'>
float64
float64
bool


In [8]:
# ---- 1. Open the NetCDF ----
ds = xr.open_dataset(r'C:\Users\taohuang\Downloads\TerraClimate_def_2024.nc')

# Inspect variable names if unsure:
# print(ds)

# ---- 2. Select the variable and its last time step ----
var = ds["def"]          # Climatic Water Deficit
last_layer = var.isel(time=-6)   # last time index

# ---- 3. Extract spatial metadata ----
data = last_layer.values

# Coordinates (assuming lat/lon regular grid)
lat = last_layer["lat"].values
lon = last_layer["lon"].values

# Compute affine transform
res_lon = lon[1] - lon[0]
res_lat = lat[1] - lat[0]

transform = from_origin(
    west=lon.min(),
    north=lat.max(),
    xsize=res_lon,
    ysize=abs(res_lat)
)

# ---- 4. Save to GeoTIFF ----
out_file = r'C:\Users\taohuang\Downloads\def_last_layer_6.tif'

with rasterio.open(
    out_file,
    "w",
    driver="GTiff",
    height=data.shape[0],
    width=data.shape[1],
    count=1,
    dtype=data.dtype,
    crs="EPSG:4326",                 # adjust if your CRS is different
    transform=transform
) as dst:
    dst.write(data, 1)

print("Saved:", out_file)


CPLE_AppDefinedError: Deleting C:\Users\taohuang\Downloads\def_last_layer_6.tif failed: Permission denied